# Automated Workflow for Processing Construction Tenders and Floor Plans

Dieses Notebook implementiert einen automatisierten Workflow zur Verarbeitung von Bauausschreibungen und Grundrissen gemäß der Projektbeschreibung im README.md.

## Workflow-Übersicht:

1. **Input Verification** - PDF-Text extrahieren (pdfminer/PyMuPDF)
2. **OCR mit Tesseract** - Falls kein Text gefunden wurde
3. **Vektorisierung des Grundrisses** - Format prüfen und ggf. vektorisieren
4. **Textanalyse & Informationsextraktion** - NLP mit spaCy
5. **Automatische Annotation** - Grundriss annotieren
6. **Erweiterungen** - Zusammenfassung, Datenbankanbindung (optional)


## Setup & Imports

Importiere alle benötigten Bibliotheken für den Workflow.

In [1]:
# Basic imports
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# PDF processing
from pdfminer.high_level import extract_text
import fitz  # PyMuPDF
from pdf2image import convert_from_path
import pytesseract

# Image processing
import cv2

# Vector graphics
import svgwrite

# NLP
import spacy

# Set Tesseract path for Windows
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Display settings
get_ipython().run_line_magic('matplotlib', 'inline')

print("Alle Bibliotheken erfolgreich importiert!")


Alle Bibliotheken erfolgreich importiert!


## 1. Input Verification

Überprüfe, ob die PDF bereits durchsuchbar ist und extrahiere Text mit pdfminer.six oder PyMuPDF.


In [2]:
def extract_text_from_pdf(pdf_path, method='auto'):
    """
    Extrahiert Text aus einer PDF-Datei mit verschiedenen Methoden.
    
    Args:
        pdf_path (str): Pfad zur PDF-Datei
        method (str): 'auto', 'pdfminer', 'pymupdf' oder 'ocr'
        
    Returns:
        tuple: (extracted_text, method_used)
    """
    
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"PDF-Datei nicht gefunden: {pdf_path}")
    
    # Method 1: pdfminer.six
    if method in ['auto', 'pdfminer']:
        try:
            print("Versuche Textextraktion mit pdfminer.six...")
            text = extract_text(pdf_path)
            if text.strip():  # Check if meaningful text was extracted
                print(f"✓ Text erfolgreich extrahiert ({len(text)} Zeichen)")
                return text, 'pdfminer'
            else:
                print("⚠ Kein Text mit pdfminer gefunden")
        except Exception as e:
            print(f"⚠ pdfminer Fehler: {e}")
    
    # Method 2: PyMuPDF
    if method in ['auto', 'pymupdf']:
        try:
            print("Versuche Textextraktion mit PyMuPDF...")
            doc = fitz.open(pdf_path)
            text = ""
            for page_num in range(len(doc)):
                page = doc[page_num]
                text += page.get_text()
            doc.close()
            
            if text.strip():
                print(f"✓ Text erfolgreich extrahiert ({len(text)} Zeichen)")
                return text, 'pymupdf'
            else:
                print("⚠ Kein Text mit PyMuPDF gefunden")
        except Exception as e:
            print(f"⚠ PyMuPDF Fehler: {e}")
    
    # Method 3: OCR fallback
    if method in ['auto', 'ocr']:
        print("Fallback: OCR wird verwendet...")
        return extract_text_with_ocr(pdf_path), 'ocr'
    
    return "", 'none'

# Test der Funktion (mit Platzhalter)
text, method = extract_text_from_pdf("example.pdf")
print(f"Verwendete Methode: {method}")
print(f"Extrahierter Text (erste 200 Zeichen): {text[:200]}...")


CropBox missing from /Page, defaulting to MediaBox


Versuche Textextraktion mit pdfminer.six...
✓ Text erfolgreich extrahiert (3121 Zeichen)
Verwendete Methode: pdfminer
Extrahierter Text (erste 200 Zeichen): Sample PDF
This is a simple PDF ﬁle. Fun fun fun.

Lorem ipsum dolor  sit amet,  consectetuer  adipiscing elit.  Phasellus  facilisis odio  sed mi. 
Curabitur suscipit. Nullam vel nisi. Etiam semper i...


## 2. OCR mit Tesseract

Konvertiere PDF-Seiten zu Bildern und extrahiere Text mit Tesseract OCR.


In [3]:
def extract_text_with_ocr(pdf_path, language='deu+eng'):
    """
    Extrahiert Text aus PDF mit OCR (Tesseract).
    
    Args:
        pdf_path (str): Pfad zur PDF-Datei
        language (str): Tesseract Sprachcode (deu=Deutsch, eng=Englisch)
        
    Returns:
        str: Extrahierter Text
    """
    
    try:
        print("Konvertiere PDF zu Bildern...")
        # PDF zu Bildern konvertieren
        images = convert_from_path(pdf_path, dpi=300)  # Höhere DPI für bessere OCR
        
        extracted_text = ""
        
        print(f"Verarbeite {len(images)} Seiten mit OCR...")
        for i, image in enumerate(images):
            print(f"  Seite {i+1}/{len(images)}")
            
            # OCR auf jedes Bild anwenden
            page_text = pytesseract.image_to_string(
                image, 
                lang=language,
                config='--psm 1'  # Page segmentation mode
            )
            
            extracted_text += f"\n--- Seite {i+1} ---\n{page_text}\n"
        
        print(f"✓ OCR abgeschlossen ({len(extracted_text)} Zeichen extrahiert)")
        return extracted_text
        
    except Exception as e:
        print(f"❌ OCR Fehler: {e}")
        return ""


In [4]:
def preprocess_image_for_ocr(image):
    """
    Verbessert ein Bild für bessere OCR-Ergebnisse.
    
    Args:
        image: PIL Image oder numpy array
        
    Returns:
        Processed image
    """
    
    # Konvertiere zu numpy array falls nötig
    if hasattr(image, 'mode'):
        image = np.array(image)
    
    # Zu Graustufen konvertieren
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    else:
        gray = image
    
    # Rauschen reduzieren
    denoised = cv2.medianBlur(gray, 3)
    
    # Kontrast verbessern
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(denoised)
    
    return enhanced

# Test der OCR-Funktion (mit Platzhalter)
ocr_text = extract_text_with_ocr("example.pdf")
print(f"OCR Text (erste 200 Zeichen): {ocr_text[:200]}...")


Konvertiere PDF zu Bildern...
Verarbeite 1 Seiten mit OCR...
  Seite 1/1
✓ OCR abgeschlossen (2872 Zeichen extrahiert)
OCR Text (erste 200 Zeichen): 
--- Seite 1 ---
sample PDF

This ıs a simple PDF file. Fun fun fun.

Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Phasellus facilisis odio sed mi.
Curabitur suscipit. Nullam vel nisi. Et...


## 3. Vektorisierung des Grundrisses

Überprüfe das Dateiformat des Grundrisses und vektorisiere bei Bedarf mit OpenCV und Potrace.


In [5]:
def check_file_format(file_path):
    """
    Überprüft das Dateiformat einer Datei.
    
    Args:
        file_path (str): Pfad zur Datei
        
    Returns:
        str: Dateiformat ('pdf', 'svg', 'dxf', 'image', 'unknown')
    """
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Datei nicht gefunden: {file_path}")
    
    # Get file extension
    _, ext = os.path.splitext(file_path)
    ext = ext.lower()
    
    format_mapping = {
        '.pdf': 'pdf',
        '.svg': 'svg',
        '.dxf': 'dxf',
        '.png': 'image',
        '.jpg': 'image',
        '.jpeg': 'image',
        '.tiff': 'image',
        '.tif': 'image',
        '.bmp': 'image'
    }
    
    return format_mapping.get(ext, 'unknown')


In [6]:
def vectorize_floorplan(file_path, output_dir="output"):
    """
    Vektorisiert einen Grundriss falls nötig.
    
    Args:
        file_path (str): Pfad zur Grundriss-Datei
        output_dir (str): Ausgabeverzeichnis
        
    Returns:
        str: Pfad zur vektorisierten SVG-Datei
    """
    
    # Ausgabeverzeichnis erstellen
    os.makedirs(output_dir, exist_ok=True)
    
    # Dateiformat überprüfen
    file_format = check_file_format(file_path)
    print(f"Erkanntes Dateiformat: {file_format}")
    
    # Base filename for output
    base_name = os.path.splitext(os.path.basename(file_path))[0]
    svg_output_path = os.path.join(output_dir, f"{base_name}_vectorized.svg")
    
    # Bereits SVG?
    if file_format == 'svg':
        print("✓ Datei ist bereits im SVG-Format")
        return file_path
    
    # PDF mit Vektorpfaden?
    elif file_format == 'pdf':
        if check_pdf_for_vectors(file_path):
            print("✓ PDF enthält Vektorpfade")
            # TODO: PDF Vektoren zu SVG konvertieren
            # Für jetzt: Fallback zu Rasterbild-Vektorisierung
            return vectorize_raster_to_svg(file_path, svg_output_path)
        else:
            print("⚠ PDF enthält keine Vektorpfade - verwende Rasterbild-Vektorisierung")
            return vectorize_raster_to_svg(file_path, svg_output_path)
    
    # Rasterbild
    elif file_format == 'image':
        print("ℹ Rasterbild erkannt - starte Vektorisierung")
        return vectorize_raster_to_svg(file_path, svg_output_path)
    
    else:
        raise ValueError(f"Nicht unterstütztes Dateiformat: {file_format}")

def check_pdf_for_vectors(pdf_path):
    """
    Überprüft, ob eine PDF-Datei Vektorpfade enthält.
    
    Args:
        pdf_path (str): Pfad zur PDF-Datei
        
    Returns:
        bool: True wenn Vektorpfade gefunden wurden
    """
    
    try:
        doc = fitz.open(pdf_path)
        for page_num in range(min(3, len(doc))):  # Nur erste 3 Seiten prüfen
            page = doc[page_num]
            paths = page.get_drawings()
            if paths:
                doc.close()
                return True
        doc.close()
        return False
    except Exception as e:
        print(f"Fehler beim Überprüfen der PDF-Vektoren: {e}")
        return False


In [8]:
# ===== HILFSFUNKTIONEN FÜR VERBESSERTE VEKTORISIERUNG =====

def detect_text_regions(image):
    """
    Erkennt Textbereiche im Bild um sie von der Linienvektorisierung auszuschließen.
    
    Args:
        image: Original-Bild (farbig oder grau)
        
    Returns:
        Binary mask der Textbereiche oder None
    """
    try:
        # Zu Graustufen konvertieren
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
        
        # MSER (Maximally Stable Extremal Regions) für Text-Erkennung
        mser = cv2.MSER_create(
            _min_area=10,      # Mindestfläche
            _max_area=5000,    # Maximale Fläche
            _delta=3           # Empfindlichkeit
        )
        
        regions, _ = mser.detectRegions(gray)
        
        # Maske erstellen
        text_mask = np.zeros(gray.shape, dtype=np.uint8)
        
        for region in regions:
            # Bounding Box der Region
            x, y, w, h = cv2.boundingRect(region.reshape(-1, 1, 2))
            
            # Filter: Textähnliche Dimensionen
            aspect_ratio = w / h if h > 0 else 0
            if 0.1 < aspect_ratio < 10 and 5 < w < 200 and 5 < h < 50:
                # Region als Text markieren
                cv2.rectangle(text_mask, (x-2, y-2), (x+w+2, y+h+2), 255, -1)
        
        # Morphologische Operationen um Text-Regionen zu verbinden
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 2))
        text_mask = cv2.morphologyEx(text_mask, cv2.MORPH_CLOSE, kernel)
        
        return text_mask
        
    except Exception as e:
        print(f"  Warnung: Text-Erkennung fehlgeschlagen: {e}")
        return None

def detect_wall_regions(binary_image):
    """
    Erkennt dicke schwarze Bereiche als Wände.
    
    Args:
        binary_image: Binäres Eingabebild
        
    Returns:
        Binary mask der Wandbereiche oder None
    """
    try:
        # Morphologische Operationen um dicke Linien zu erkennen
        # Horizontale Strukturen
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 3))
        horizontal_walls = cv2.morphologyEx(binary_image, cv2.MORPH_OPEN, horizontal_kernel)
        
        # Vertikale Strukturen  
        vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 15))
        vertical_walls = cv2.morphologyEx(binary_image, cv2.MORPH_OPEN, vertical_kernel)
        
        # Kombiniere horizontale und vertikale Wände
        wall_regions = cv2.bitwise_or(horizontal_walls, vertical_walls)
        
        # Zusätzliche Filterung: Entferne sehr kleine Bereiche
        contours, _ = cv2.findContours(wall_regions, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        filtered_walls = np.zeros_like(wall_regions)
        
        for contour in contours:
            area = cv2.contourArea(contour)
            if area > 100:  # Mindestfläche für Wände
                cv2.fillPoly(filtered_walls, [contour], 255)
        
        return filtered_walls
        
    except Exception as e:
        print(f"  Warnung: Wand-Erkennung fehlgeschlagen: {e}")
        return None

def detect_and_merge_lines(image):
    """
    Intelligente Linienerkennung mit Verschmelzung ähnlicher Linien.
    
    Args:
        image: Bereinigtes binäres Bild
        
    Returns:
        Liste verschmolzener Linien
    """
    try:
        # Hough-Linien erkennen mit strengeren Parametern
        lines = cv2.HoughLinesP(
            image, 
            rho=1, 
            theta=np.pi/180, 
            threshold=30,        # Höhere Schwelle
            minLineLength=20,    # Mindestlänge
            maxLineGap=5         # Kleinere Lücken
        )
        
        if lines is None or len(lines) == 0:
            return []
        
        # Linien in einfacheres Format konvertieren
        line_list = []
        for line in lines:
            x1, y1, x2, y2 = line[0]
            line_list.append((x1, y1, x2, y2))
        
        # Linien nach Ähnlichkeit gruppieren und verschmelzen
        merged_lines = merge_similar_lines(line_list)
        
        print(f"  Linien: {len(line_list)} → {len(merged_lines)} (nach Verschmelzung)")
        return merged_lines
        
    except Exception as e:
        print(f"  Warnung: Linienerkennung fehlgeschlagen: {e}")
        return []

def merge_similar_lines(lines, angle_threshold=5, distance_threshold=10):
    """
    Verschmilzt ähnliche/kollineare Linien.
    
    Args:
        lines: Liste von Linien (x1, y1, x2, y2)
        angle_threshold: Winkel-Toleranz in Grad
        distance_threshold: Abstand-Toleranz in Pixeln
        
    Returns:
        Liste verschmolzener Linien
    """
    if not lines:
        return []
    
    merged = []
    used = set()
    
    for i, line1 in enumerate(lines):
        if i in used:
            continue
            
        x1, y1, x2, y2 = line1
        
        # Suche ähnliche Linien
        similar_lines = [line1]
        used.add(i)
        
        for j, line2 in enumerate(lines):
            if j in used or i == j:
                continue
                
            if are_lines_similar(line1, line2, angle_threshold, distance_threshold):
                similar_lines.append(line2)
                used.add(j)
        
        # Verschmelze ähnliche Linien zu einer
        if len(similar_lines) > 1:
            merged_line = merge_line_group(similar_lines)
            merged.append(merged_line)
        else:
            merged.append(line1)
    
    return merged

def are_lines_similar(line1, line2, angle_threshold, distance_threshold):
    """
    Überprüft ob zwei Linien ähnlich sind (ähnlicher Winkel und nahe beieinander).
    """
    x1, y1, x2, y2 = line1
    x3, y3, x4, y4 = line2
    
    # Winkel berechnen
    angle1 = np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi
    angle2 = np.arctan2(y4 - y3, x4 - x3) * 180 / np.pi
    
    # Winkel-Differenz (berücksichtige 180°-Periodizität)
    angle_diff = abs(angle1 - angle2)
    angle_diff = min(angle_diff, 180 - angle_diff)
    
    if angle_diff > angle_threshold:
        return False
    
    # Abstand zwischen Linien berechnen
    # Vereinfacht: Abstand zwischen Mittelpunkten
    mid1 = ((x1 + x2) / 2, (y1 + y2) / 2)
    mid2 = ((x3 + x4) / 2, (y3 + y4) / 2)
    distance = np.sqrt((mid1[0] - mid2[0])**2 + (mid1[1] - mid2[1])**2)
    
    return distance < distance_threshold

def merge_line_group(lines):
    """
    Verschmilzt eine Gruppe ähnlicher Linien zu einer einzigen Linie.
    """
    # Sammle alle Endpunkte
    points = []
    for x1, y1, x2, y2 in lines:
        points.extend([(x1, y1), (x2, y2)])
    
    # Finde die beiden äußersten Punkte
    points = np.array(points)
    
    # Hauptachse der Punkte finden (PCA)
    mean_point = np.mean(points, axis=0)
    centered_points = points - mean_point
    
    # SVD für Hauptrichtung
    _, _, vh = np.linalg.svd(centered_points)
    direction = vh[0]
    
    # Projiziere alle Punkte auf die Hauptachse
    projections = np.dot(centered_points, direction)
    
    # Äußerste Projektionen finden
    min_proj = np.min(projections)
    max_proj = np.max(projections)
    
    # Zurück zu Koordinaten
    start_point = mean_point + min_proj * direction
    end_point = mean_point + max_proj * direction
    
    return (int(start_point[0]), int(start_point[1]), 
            int(end_point[0]), int(end_point[1]))


In [9]:
# ===== SVG-ERSTELLUNGSFUNKTIONEN =====

def add_wall_hatches(dwg, wall_regions, width, height):
    """
    Fügt Wände als Schraffuren zum SVG hinzu.
    
    Args:
        dwg: SVG Drawing Objekt
        wall_regions: Binary mask der Wandbereiche
        width, height: Bildabmessungen
        
    Returns:
        Anzahl der hinzugefügten Wände
    """
    if wall_regions is None:
        return 0
    
    try:
        # Schraffur-Pattern definieren
        pattern_id = "wallHatch"
        pattern = dwg.defs.add(dwg.pattern(
            id=pattern_id,
            patternUnits="userSpaceOnUse",
            size=(6, 6)
        ))
        
        # Diagonale Linien für Schraffur
        pattern.add(dwg.line(start=(0, 0), end=(6, 6), stroke="black", stroke_width="0.5"))
        pattern.add(dwg.line(start=(0, 6), end=(6, 0), stroke="black", stroke_width="0.5"))
        
        # Wandkonturen finden
        contours, _ = cv2.findContours(wall_regions, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        wall_count = 0
        
        for contour in contours:
            area = cv2.contourArea(contour)
            if area > 50:  # Nur größere Wände
                # Kontur vereinfachen
                epsilon = 0.02 * cv2.arcLength(contour, True)
                approx = cv2.approxPolyDP(contour, epsilon, True)
                
                if len(approx) >= 3:
                    # Path für SVG erstellen
                    path_data = f"M {approx[0][0][0]},{approx[0][0][1]}"
                    for point in approx[1:]:
                        path_data += f" L {point[0][0]},{point[0][1]}"
                    path_data += " Z"
                    
                    # Wand mit Schraffur hinzufügen
                    dwg.add(dwg.path(d=path_data, 
                                   fill=f"url(#{pattern_id})",
                                   stroke="black", 
                                   stroke_width="2"))
                    wall_count += 1
        
        return wall_count
        
    except Exception as e:
        print(f"  Warnung: Wand-Schraffuren fehlgeschlagen: {e}")
        return 0

def add_lines_to_svg(dwg, lines):
    """
    Fügt bereinigte Linien zum SVG hinzu.
    
    Args:
        dwg: SVG Drawing Objekt
        lines: Liste von Linien (x1, y1, x2, y2)
        
    Returns:
        Anzahl der hinzugefügten Linien
    """
    line_count = 0
    
    for line in lines:
        x1, y1, x2, y2 = line
        
        # Linenlänge prüfen
        length = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        if length > 10:  # Mindestlänge
            dwg.add(dwg.line(
                start=(int(x1), int(y1)), 
                end=(int(x2), int(y2)),
                stroke="black", 
                stroke_width="1",
                stroke_linecap="round"
            ))
            line_count += 1
    
    return line_count

def detect_important_contours(image, width, height):
    """
    Erkennt wichtige Konturen ohne Überlappungen.
    
    Args:
        image: Bereinigtes binäres Bild
        width, height: Bildabmessungen
        
    Returns:
        Liste wichtiger Konturen
    """
    try:
        # Konturen finden mit Hierarchie
        contours, hierarchy = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        important_contours = []
        min_area = (width * height) * 0.0001  # 0.01% der Gesamtfläche
        max_area = (width * height) * 0.5     # 50% der Gesamtfläche
        
        for i, contour in enumerate(contours):
            area = cv2.contourArea(contour)
            perimeter = cv2.arcLength(contour, True)
            
            # Filter nach Fläche
            if not (min_area < area < max_area):
                continue
            
            # Filter nach Seitenverhältnis (vermeidet sehr schmale Objekte)
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = max(w, h) / min(w, h) if min(w, h) > 0 else float('inf')
            if aspect_ratio > 20:  # Zu schmale Objekte ausschließen
                continue
            
            # Filter nach Kompaktheit (vermeidet sehr verzweigte Formen)
            if perimeter > 0:
                compactness = 4 * np.pi * area / (perimeter * perimeter)
                if compactness < 0.01:  # Zu verzweigte Formen ausschließen
                    continue
            
            # Äußere Konturen bevorzugen (Hierarchie prüfen)
            if hierarchy is not None:
                parent = hierarchy[0][i][3]
                if parent == -1 or len(important_contours) < 50:  # Äußere Konturen oder Limit
                    important_contours.append(contour)
            else:
                important_contours.append(contour)
        
        return important_contours[:100]  # Maximal 100 Konturen
        
    except Exception as e:
        print(f"  Warnung: Kontur-Erkennung fehlgeschlagen: {e}")
        return []

def add_contours_to_svg(dwg, contours):
    """
    Fügt wichtige Konturen zum SVG hinzu.
    
    Args:
        dwg: SVG Drawing Objekt
        contours: Liste von OpenCV Konturen
        
    Returns:
        Anzahl der hinzugefügten Konturen
    """
    contour_count = 0
    
    for contour in contours:
        # Kontur vereinfachen
        epsilon = 0.01 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        
        if len(approx) >= 3:
            # Path erstellen
            path_data = f"M {approx[0][0][0]},{approx[0][0][1]}"
            for point in approx[1:]:
                path_data += f" L {point[0][0]},{point[0][1]}"
            path_data += " Z"
            
            dwg.add(dwg.path(d=path_data, 
                           fill="none", 
                           stroke="gray", 
                           stroke_width="0.5",
                           stroke_dasharray="2,2"))
            contour_count += 1
    
    return contour_count

def add_text_to_svg(dwg, image, text_mask):
    """
    Extrahiert Text mit OCR und fügt ihn als echten Text zum SVG hinzu.
    
    Args:
        dwg: SVG Drawing Objekt
        image: Original-Bild
        text_mask: Maske der Textbereiche
        
    Returns:
        Anzahl der hinzugefügten Texte
    """
    if text_mask is None:
        return 0
    
    try:
        # Textbereiche finden
        contours, _ = cv2.findContours(text_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        text_count = 0
        
        for contour in contours:
            # Bounding Box der Textregion
            x, y, w, h = cv2.boundingRect(contour)
            
            # Mindestgröße für Text
            if w < 10 or h < 5:
                continue
            
            # Textregion aus Bild ausschneiden
            text_region = image[y:y+h, x:x+w]
            
            try:
                # OCR auf die Textregion anwenden
                text_content = pytesseract.image_to_string(
                    text_region, 
                    lang='deu+eng',
                    config='--psm 8 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyzäöüÄÖÜß0123456789.,- '
                ).strip()
                
                # Nur hinzufügen wenn sinnvoller Text erkannt wurde
                if len(text_content) > 1 and any(c.isalnum() for c in text_content):
                    # Schriftgröße basierend auf Texthöhe schätzen
                    font_size = max(8, min(16, h * 0.7))
                    
                    # Text zum SVG hinzufügen
                    dwg.add(dwg.text(
                        text_content,
                        insert=(x, y + h * 0.8),  # Baseline-korrigierte Position
                        font_size=f"{font_size}px",
                        font_family="Arial, sans-serif",
                        fill="blue"
                    ))
                    text_count += 1
                    
            except Exception as e:
                # Einzelne OCR-Fehler ignorieren
                continue
        
        return text_count
        
    except Exception as e:
        print(f"  Warnung: Text-Extraktion fehlgeschlagen: {e}")
        return 0


In [11]:
def vectorize_raster_to_svg(image_path, output_path):
    """
    Verbesserte Konvertierung eines Rasterbilds zu SVG für Architekturpläne.
    
    Features:
    - Text-Erkennung und -Erhaltung
    - Intelligente Linienverschmelzung
    - Wand-Schraffuren für dicke schwarze Bereiche
    - Reduzierung überlappender Geometrien
    
    Args:
        image_path (str): Pfad zum Eingabebild
        output_path (str): Pfad für die Ausgabe-SVG
        
    Returns:
        str: Pfad zur erstellten SVG-Datei
    """
    
    try:
        # Lade das Bild
        if image_path.endswith('.pdf'):
            images = convert_from_path(image_path, dpi=300)
            image = np.array(images[0])
        else:
            image = cv2.imread(image_path)
        
        if image is None:
            raise ValueError("Bild konnte nicht geladen werden")
        
        # Original für Text-Erkennung behalten
        original_image = image.copy()
        
        # Bildverarbeitung für technische Zeichnungen
        processed_image = preprocess_for_architectural_drawings(image)
        height, width = processed_image.shape
        
        # 1. TEXT-ERKENNUNG UND -FILTER
        print("Erkenne Textbereiche...")
        text_mask = detect_text_regions(original_image)
        
        # 2. WAND-ERKENNUNG (dicke schwarze Bereiche)
        print("Erkenne Wände...")
        wall_regions = detect_wall_regions(processed_image)
        
        # 3. BEREINIGTES BILD FÜR LINIENERKENNUNG
        # Entferne Text und Wände aus der Linienerkennung
        lines_image = processed_image.copy()
        if text_mask is not None:
            lines_image = cv2.bitwise_and(lines_image, cv2.bitwise_not(text_mask))
        if wall_regions is not None:
            lines_image = cv2.bitwise_and(lines_image, cv2.bitwise_not(wall_regions))
        
        # SVG erstellen
        dwg = svgwrite.Drawing(output_path, size=(f"{width}px", f"{height}px"))
        
        # 4. WÄNDE ALS SCHRAFFUREN HINZUFÜGEN
        wall_count = add_wall_hatches(dwg, wall_regions, width, height)
        
        # 5. INTELLIGENTE LINIENERKENNUNG
        print("Erkenne und verschmelze Linien...")
        cleaned_lines = detect_and_merge_lines(lines_image)
        line_count = add_lines_to_svg(dwg, cleaned_lines)
        
        # 6. WICHTIGE KONTUREN (ohne Überlappungen)
        print("Erkenne wichtige Konturen...")
        important_contours = detect_important_contours(lines_image, width, height)
        contour_count = add_contours_to_svg(dwg, important_contours)
        
        # 7. TEXT HINZUFÜGEN
        print("Extrahiere und füge Text hinzu...")
        text_count = add_text_to_svg(dwg, original_image, text_mask)
        
        # SVG speichern
        dwg.save()
        print(f"✓ Verbessertes SVG erstellt: {output_path}")
        print(f"  Wände: {wall_count}, Linien: {line_count}, Konturen: {contour_count}, Texte: {text_count}")
        return output_path
        
    except Exception as e:
        print(f"❌ Fehler bei der verbesserten Vektorisierung: {e}")
        return None

def preprocess_for_architectural_drawings(image):
    """
    Spezielle Bildvorverarbeitung für Architekturzeichnungen und technische Pläne.
    
    Args:
        image: Eingabebild (numpy array)
        
    Returns:
        Vorverarbeitetes binäres Bild
    """
    
    # Zu Graustufen konvertieren
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image
    
    # Kontrast verbessern
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    enhanced = clahe.apply(gray)
    
    # Leichter Gaußscher Weichzeichner um Rauschen zu reduzieren
    blurred = cv2.GaussianBlur(enhanced, (3, 3), 0)
    
    # Zwei verschiedene Schwellenwert-Methoden kombinieren
    # 1. Adaptive Schwellenwert für lokale Variationen
    adaptive = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                   cv2.THRESH_BINARY, 11, 2)
    
    # 2. Otsu-Schwellenwert für globale Binarisierung
    _, otsu = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Kombiniere beide Methoden
    combined = cv2.bitwise_and(adaptive, otsu)
    
    # Morphologische Operationen um Linien zu verbessern
    # Kleiner Kernel für feine Details
    kernel_small = np.ones((2,2), np.uint8)
    # Größerer Kernel für Linien
    kernel_line = cv2.getStructuringElement(cv2.MORPH_RECT, (3,1))
    
    # Schließe kleine Lücken in Linien
    closed = cv2.morphologyEx(combined, cv2.MORPH_CLOSE, kernel_small)
    
    # Entferne kleine Artefakte
    opened = cv2.morphologyEx(closed, cv2.MORPH_OPEN, kernel_small)
    
    # Invertiere falls nötig (schwarze Linien auf weißem Hintergrund)
    # Überprüfe welche Farbe mehr Pixel hat
    if np.sum(opened == 0) < np.sum(opened == 255):
        opened = cv2.bitwise_not(opened)
    
    return opened

# Test der Vektorisierungsfunktion (mit Platzhalter)
svg_path = vectorize_floorplan("example_floorplan2.pdf")
print(f"Vektorisierter Grundriss: {svg_path}")


Erkanntes Dateiformat: pdf
⚠ PDF enthält keine Vektorpfade - verwende Rasterbild-Vektorisierung
Erkenne Textbereiche...
  Warnung: Text-Erkennung fehlgeschlagen: '_min_area' is an invalid keyword argument for MSER_create()
Erkenne Wände...
Erkenne und verschmelze Linien...
  Linien: 815 → 387 (nach Verschmelzung)
Erkenne wichtige Konturen...
Extrahiere und füge Text hinzu...
✓ Verbessertes SVG erstellt: output\example_floorplan2_vectorized.svg
  Wände: 107, Linien: 387, Konturen: 24, Texte: 0
Vektorisierter Grundriss: output\example_floorplan2_vectorized.svg


In [12]:
# ===== TEST DER VERBESSERTEN VEKTORISIERUNG =====

print("🚀 === TEST DER VERBESSERTEN VEKTORISIERUNG ===")
print()

# Test mit verfügbarem Grundriss
test_files = ["example_floorplan.pdf", "example_floorplan2.pdf"]

for test_file in test_files:
    if os.path.exists(test_file):
        print(f"📄 Teste mit: {test_file}")
        print("-" * 50)
        
        try:
            # Verbesserte Vektorisierung ausführen
            svg_path = vectorize_floorplan(test_file, "output_improved")
            
            if svg_path:
                print(f"✅ Verbessertes SVG erstellt: {svg_path}")
                
                # Dateigröße prüfen
                if os.path.exists(svg_path):
                    file_size = os.path.getsize(svg_path) / 1024  # KB
                    print(f"📊 Dateigröße: {file_size:.1f} KB")
                
            else:
                print("❌ Vektorisierung fehlgeschlagen")
                
        except Exception as e:
            print(f"❌ Fehler: {e}")
        
        print()
    else:
        print(f"⚠️ Datei nicht gefunden: {test_file}")

print("💡 VERBESSERUNGEN:")
print("✓ Text-Erkennung verhindert Übervektorisierung von Beschriftungen")
print("✓ Wände werden als Schraffuren dargestellt")  
print("✓ Intelligente Linienverschmelzung reduziert Überlappungen")
print("✓ Wichtige Konturen werden gefiltert")
print("✓ OCR-extrahierter Text wird als echte SVG-Texte eingefügt")
print()
print("🎯 NÄCHSTE SCHRITTE:")
print("• Parameter für spezifische Grundriss-Typen anpassen")
print("• Fein-Tuning der Filter-Schwellenwerte") 
print("• Integration von Architektur-spezifischen Symbolen")
print("• Export-Optionen für CAD-Software")


🚀 === TEST DER VERBESSERTEN VEKTORISIERUNG ===

📄 Teste mit: example_floorplan.pdf
--------------------------------------------------
Erkanntes Dateiformat: pdf
⚠ PDF enthält keine Vektorpfade - verwende Rasterbild-Vektorisierung
Erkenne Textbereiche...
  Warnung: Text-Erkennung fehlgeschlagen: '_min_area' is an invalid keyword argument for MSER_create()
Erkenne Wände...
Erkenne und verschmelze Linien...
  Linien: 104 → 52 (nach Verschmelzung)
Erkenne wichtige Konturen...
Extrahiere und füge Text hinzu...
✓ Verbessertes SVG erstellt: output_improved\example_floorplan_vectorized.svg
  Wände: 30, Linien: 52, Konturen: 0, Texte: 0
✅ Verbessertes SVG erstellt: output_improved\example_floorplan_vectorized.svg
📊 Dateigröße: 10.4 KB

📄 Teste mit: example_floorplan2.pdf
--------------------------------------------------
Erkanntes Dateiformat: pdf
⚠ PDF enthält keine Vektorpfade - verwende Rasterbild-Vektorisierung
Erkenne Textbereiche...
  Warnung: Text-Erkennung fehlgeschlagen: '_min_area' is

In [ ]:
# ===== HILFSFUNKTION FÜR PDF-ANALYSE =====

def is_pdf_searchable(pdf_path):
    """
    Überprüft, ob eine PDF-Datei durchsuchbar ist (Text enthält).
    
    Args:
        pdf_path (str): Pfad zur PDF-Datei
        
    Returns:
        bool: True wenn durchsuchbar, False wenn nur Bilder
    """
    try:
        # Methode 1: pdfminer versuchen
        text = extract_text(pdf_path)
        if text and len(text.strip()) > 10:  # Mindestens 10 Zeichen sinnvoller Text
            print(f"📄 PDF-Status: Durchsuchbar")
            print(f"📝 Gefundener Text: {len(text)} Zeichen")
            return True
        
        # Methode 2: PyMuPDF versuchen
        doc = fitz.open(pdf_path)
        total_text = ""
        for page_num in range(min(3, len(doc))):  # Erste 3 Seiten prüfen
            page = doc[page_num]
            page_text = page.get_text()
            total_text += page_text
        doc.close()
        
        if len(total_text.strip()) > 10:
            print(f"📄 PDF-Status: Durchsuchbar")
            print(f"📝 Gefundener Text: {len(total_text)} Zeichen")
            return True
        else:
            print(f"📄 PDF-Status: Nicht durchsuchbar (Raster-PDF)")
            return False
            
    except Exception as e:
        print(f"⚠️ Fehler beim Überprüfen der PDF: {e}")
        return False


In [52]:
# Test mit der korrigierten Vektorisierung
print("=== Test der korrigierten Vektorisierung ===")
try:
    svg_path = vectorize_floorplan("example_floorplan.pdf")
    if svg_path:
        print(f"✅ Vektorisierung erfolgreich: {svg_path}")
    else:
        print("❌ Vektorisierung fehlgeschlagen")
except Exception as e:
    print(f"❌ Unerwarteter Fehler: {e}")


=== Test der korrigierten Vektorisierung ===
Erkanntes Dateiformat: pdf
⚠ PDF enthält keine Vektorpfade - verwende Rasterbild-Vektorisierung
Erkenne Konturen...
Erkenne gerade Linien...
Erkenne Kreise...
✓ SVG erstellt: output\example_floorplan_vectorized.svg
  Konturen: 1987, Linien: 8999, Kreise: 312
✅ Vektorisierung erfolgreich: output\example_floorplan_vectorized.svg


In [ ]:
# Zusätzliche Hilfsfunktion für bessere Visualisierung und Debugging
def save_debug_images(image, processed_image, output_dir="debug"):
    """
    Speichert Debug-Bilder um die Bildverarbeitung zu visualisieren.
    """
    os.makedirs(output_dir, exist_ok=True)
    
    # Original speichern
    if len(image.shape) == 3:
        cv2.imwrite(os.path.join(output_dir, "01_original.png"), cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
    else:
        cv2.imwrite(os.path.join(output_dir, "01_original.png"), image)
    
    # Verarbeitetes Bild speichern
    cv2.imwrite(os.path.join(output_dir, "02_processed.png"), processed_image)
    
    print(f"Debug-Bilder gespeichert in: {output_dir}")

# Test mit verbesserter Vektorisierung
print("=== Test der verbesserten Vektorisierung ===")
svg_path = vectorize_floorplan("example_floorplan2.pdf")
print(f"Vektorisierter Grundriss: {svg_path}")


=== Test der verbesserten Vektorisierung ===
Erkanntes Dateiformat: pdf
⚠ PDF enthält keine Vektorpfade - verwende Rasterbild-Vektorisierung
Erkenne Konturen...
Erkenne gerade Linien...
Erkenne Kreise...


## 4. Textanalyse & Informationsextraktion

Verwende NLP (spaCy) um relevante Schlüsselwörter, Positionen, Materialien und Aufgaben zu extrahieren.


In [36]:
def extract_construction_information(text):
    """
    Extrahiert relevante Bauinformationen aus Text mit spaCy NLP.
    
    Args:
        text (str): Eingabetext aus PDF/OCR
        
    Returns:
        dict: Strukturierte Informationen (Materialien, Räume, Maße, etc.)
    """
    
    try:
        # Lade deutsches spaCy-Modell
        nlp = spacy.load('de_core_news_lg')
        
        # Text verarbeiten
        doc = nlp(text)
        
        # Initialisiere Ergebnis-Dictionary
        extracted_info = {
            'materials': [],
            'rooms': [],
            'measurements': [],
            'positions': [],
            'tasks': [],
            'entities': [],
            'keywords': []
        }
        
        # 1. Benannte Entitäten extrahieren
        for ent in doc.ents:
            extracted_info['entities'].append({
                'text': ent.text,
                'label': ent.label_,
                'description': spacy.explain(ent.label_)
            })
        
        # 2. Baumaterialien finden (einfache Keyword-Liste)
        material_keywords = [
            'beton', 'stahlbeton', 'ziegel', 'mauerwerk', 'holz', 'stahl',
            'glas', 'aluminium', 'kunststoff', 'dämm', 'isolier', 'putz',
            'fliesen', 'parkett', 'laminat', 'estrich', 'fundament'
        ]
        
        for token in doc:
            if any(keyword in token.text.lower() for keyword in material_keywords):
                extracted_info['materials'].append({
                    'text': token.text,
                    'context': token.sent.text[:100] + "..." if len(token.sent.text) > 100 else token.sent.text
                })
        
        # 3. Räume und Bereiche finden
        room_keywords = [
            'raum', 'zimmer', 'küche', 'bad', 'wohnzimmer', 'schlafzimmer',
            'flur', 'diele', 'keller', 'dachboden', 'garage', 'balkon',
            'terrasse', 'büro', 'arbeitszimmer', 'gäste'
        ]
        
        for token in doc:
            if any(keyword in token.text.lower() for keyword in room_keywords):
                extracted_info['rooms'].append({
                    'text': token.text,
                    'context': token.sent.text[:100] + "..." if len(token.sent.text) > 100 else token.sent.text
                })
        
        # 4. Maße und Dimensionen finden (einfacher Regex-Ansatz)
        import re
        measurement_patterns = [
            r'\d+[,.]?\d*\s*[x×]\s*\d+[,.]?\d*\s*m',  # "3,5 x 4,2 m"
            r'\d+[,.]?\d*\s*m²',                        # "25,3 m²"
            r'\d+[,.]?\d*\s*cm',                        # "120 cm"
            r'\d+[,.]?\d*\s*mm',                        # "800 mm"
        ]
        
        for pattern in measurement_patterns:
            matches = re.findall(pattern, text, re.IGNORECASE)
            for match in matches:
                extracted_info['measurements'].append(match)
        
        # 5. Positionen und Lage-Angaben
        position_keywords = [
            'nord', 'süd', 'ost', 'west', 'links', 'rechts', 'oben', 'unten',
            'erdgeschoss', 'obergeschoss', 'untergeschoss', 'dachgeschoss'
        ]
        
        for token in doc:
            if any(keyword in token.text.lower() for keyword in position_keywords):
                extracted_info['positions'].append({
                    'text': token.text,
                    'context': token.sent.text[:100] + "..." if len(token.sent.text) > 100 else token.sent.text
                })
        
        print(f"✓ NLP-Analyse abgeschlossen:")
        print(f"  Entitäten: {len(extracted_info['entities'])}")
        print(f"  Materialien: {len(extracted_info['materials'])}")
        print(f"  Räume: {len(extracted_info['rooms'])}")
        print(f"  Maße: {len(extracted_info['measurements'])}")
        print(f"  Positionen: {len(extracted_info['positions'])}")
        
        return extracted_info
        
    except Exception as e:
        print(f"❌ Fehler bei der NLP-Analyse: {e}")
        return {}


In [ ]:
def create_structured_dataframe(extracted_info):
    """
    Konvertiert extrahierte Informationen in strukturierte pandas DataFrames.
    
    Args:
        extracted_info (dict): Ergebnis von extract_construction_information()
        
    Returns:
        dict: Dictionary mit DataFrames für verschiedene Kategorien
    """
    
    dataframes = {}
    
    # Materials DataFrame
    if extracted_info.get('materials'):
        materials_data = []
        for item in extracted_info['materials']:
            materials_data.append({
                'Material': item['text'],
                'Kontext': item['context'],
                'Kategorie': 'Material'
            })
        dataframes['materials'] = pd.DataFrame(materials_data)
    
    # Rooms DataFrame
    if extracted_info.get('rooms'):
        rooms_data = []
        for item in extracted_info['rooms']:
            rooms_data.append({
                'Raum': item['text'],
                'Kontext': item['context'],
                'Kategorie': 'Raum'
            })
        dataframes['rooms'] = pd.DataFrame(rooms_data)
    
    # Measurements DataFrame
    if extracted_info.get('measurements'):
        measurements_data = []
        for measurement in extracted_info['measurements']:
            measurements_data.append({
                'Maß': measurement,
                'Kategorie': 'Abmessung'
            })
        dataframes['measurements'] = pd.DataFrame(measurements_data)
    
    # Entities DataFrame
    if extracted_info.get('entities'):
        entities_data = []
        for entity in extracted_info['entities']:
            entities_data.append({
                'Text': entity['text'],
                'Typ': entity['label'],
                'Beschreibung': entity['description'],
                'Kategorie': 'Entität'
            })
        dataframes['entities'] = pd.DataFrame(entities_data)
    
    return dataframes

# Test der NLP-Funktionen mit dem bereits extrahierten Text
print("=== Test der NLP-Informationsextraktion ===")
if 'text' in locals():
    extracted_info = extract_construction_information(text)
    dataframes = create_structured_dataframe(extracted_info)
    
    # Zeige Ergebnisse
    for category, df in dataframes.items():
        if not df.empty:
            print(f"\n{category.upper()}:")
            print(df.head())
else:
    print("Verwende Beispieltext für Demo...")
    example_text = """
    Die Wände des Gebäudes werden in Stahlbeton ausgeführt. 
    Das Wohnzimmer hat eine Größe von 25,3 m². 
    Die Küche wird mit Fliesen ausgestattet.
    Im Erdgeschoss befinden sich 3 Räume.
    """
    extracted_info = extract_construction_information(example_text)
    dataframes = create_structured_dataframe(extracted_info)


=== Test der NLP-Informationsextraktion ===
✓ NLP-Analyse abgeschlossen:
  Entitäten: 99
  Materialien: 0
  Räume: 0
  Maße: 0
  Positionen: 0

ENTITIES:
                            Text   Typ  \
0   Sample PDF\nThis is a simple  MISC   
1                    Fun fun fun  MISC   
2  consectetuer  adipiscing elit  MISC   
3                       suscipit   PER   
4                         Nullam   PER   

                                        Beschreibung Kategorie  
0  Miscellaneous entities, e.g. events, nationali...   Entität  
1  Miscellaneous entities, e.g. events, nationali...   Entität  
2  Miscellaneous entities, e.g. events, nationali...   Entität  
3                            Named person or family.   Entität  
4                            Named person or family.   Entität  


## 5. Automatische Annotation des Grundrisses

Lade den vektorisierten Grundriss und platziere automatisch generierte Annotationen basierend auf den extrahierten Informationen.


In [38]:
def annotate_floorplan_with_info(svg_path, extracted_info, output_path=None):
    """
    Fügt Annotationen zum SVG-Grundriss basierend auf extrahierten Informationen hinzu.
    
    Args:
        svg_path (str): Pfad zum vektorisierten SVG-Grundriss
        extracted_info (dict): Extrahierte Informationen aus NLP
        output_path (str): Ausgabepfad für annotierte SVG (optional)
        
    Returns:
        str: Pfad zur annotierten SVG-Datei
    """
    
    if not svg_path or not os.path.exists(svg_path):
        print("❌ SVG-Grundriss nicht gefunden")
        return None
    
    # Ausgabepfad bestimmen
    if not output_path:
        base_name = os.path.splitext(svg_path)[0]
        output_path = f"{base_name}_annotated.svg"
    
    try:
        # SVG-Dimensionen aus der ursprünglichen Datei lesen
        import xml.etree.ElementTree as ET
        tree = ET.parse(svg_path)
        root = tree.getroot()
        
        # Versuche Dimensionen zu extrahieren
        width = 800  # Fallback-Werte
        height = 600
        
        if 'viewBox' in root.attrib:
            viewbox = root.attrib['viewBox'].split()
            width, height = int(float(viewbox[2])), int(float(viewbox[3]))
        elif 'width' in root.attrib and 'height' in root.attrib:
            width = int(float(root.attrib['width'].replace('px', '')))
            height = int(float(root.attrib['height'].replace('px', '')))
        
        # Neue SVG mit Annotationen erstellen
        dwg = svgwrite.Drawing(output_path, size=(f"{width}px", f"{height}px"))
        
        # Lade das ursprüngliche SVG als Hintergrund
        # (Vereinfacht: kopiere nur die Grundriss-Geometrie)
        
        # Annotation-Bereiche definieren
        annotation_areas = {
            'materials': {'x': 20, 'y': 30, 'color': 'blue'},
            'rooms': {'x': 20, 'y': height//2, 'color': 'green'},
            'measurements': {'x': width-200, 'y': 30, 'color': 'red'},
            'general': {'x': width-200, 'y': height//2, 'color': 'purple'}
        }
        
        # Titel hinzufügen
        dwg.add(dwg.text("Automatisch annotierter Grundriss", 
                        insert=(width//2, 20), 
                        text_anchor="middle", 
                        font_size="16px", 
                        font_weight="bold"))
        
        # Materialien annotieren
        if extracted_info.get('materials'):
            y_pos = annotation_areas['materials']['y']
            dwg.add(dwg.text("Materialien:", 
                           insert=(annotation_areas['materials']['x'], y_pos), 
                           font_size="14px", 
                           font_weight="bold", 
                           fill=annotation_areas['materials']['color']))
            
            for i, material in enumerate(extracted_info['materials'][:5]):  # Max 5 Einträge
                y_pos += 20
                dwg.add(dwg.text(f"• {material['text']}", 
                               insert=(annotation_areas['materials']['x'] + 10, y_pos), 
                               font_size="12px", 
                               fill=annotation_areas['materials']['color']))
        
        # Räume annotieren
        if extracted_info.get('rooms'):
            y_pos = annotation_areas['rooms']['y']
            dwg.add(dwg.text("Räume:", 
                           insert=(annotation_areas['rooms']['x'], y_pos), 
                           font_size="14px", 
                           font_weight="bold", 
                           fill=annotation_areas['rooms']['color']))
            
            for i, room in enumerate(extracted_info['rooms'][:5]):  # Max 5 Einträge
                y_pos += 20
                dwg.add(dwg.text(f"• {room['text']}", 
                               insert=(annotation_areas['rooms']['x'] + 10, y_pos), 
                               font_size="12px", 
                               fill=annotation_areas['rooms']['color']))
        
        # Maße annotieren
        if extracted_info.get('measurements'):
            y_pos = annotation_areas['measurements']['y']
            dwg.add(dwg.text("Abmessungen:", 
                           insert=(annotation_areas['measurements']['x'], y_pos), 
                           font_size="14px", 
                           font_weight="bold", 
                           fill=annotation_areas['measurements']['color']))
            
            for i, measurement in enumerate(extracted_info['measurements'][:5]):  # Max 5 Einträge
                y_pos += 20
                dwg.add(dwg.text(f"• {measurement}", 
                               insert=(annotation_areas['measurements']['x'] + 10, y_pos), 
                               font_size="12px", 
                               fill=annotation_areas['measurements']['color']))
        
        # Legende hinzufügen
        legend_y = height - 100
        dwg.add(dwg.text("Legende:", 
                       insert=(20, legend_y), 
                       font_size="14px", 
                       font_weight="bold"))
        
        legend_items = [
            ("Materialien", annotation_areas['materials']['color']),
            ("Räume", annotation_areas['rooms']['color']),
            ("Abmessungen", annotation_areas['measurements']['color'])
        ]
        
        for i, (label, color) in enumerate(legend_items):
            y = legend_y + 20 + (i * 20)
            dwg.add(dwg.circle(center=(30, y-5), r=5, fill=color))
            dwg.add(dwg.text(label, insert=(45, y), font_size="12px"))
        
        # SVG speichern
        dwg.save()
        print(f"✓ Annotierter Grundriss erstellt: {output_path}")
        return output_path
        
    except Exception as e:
        print(f"❌ Fehler bei der Annotation: {e}")
        return None


## 6. Kompletter Workflow

Zusammenführung aller Schritte in einer einzigen Workflow-Funktion.


In [39]:
def process_construction_documents(pdf_path, floorplan_path, output_dir="workflow_output"):
    """
    Kompletter automatisierter Workflow für Bauausschreibungen und Grundrisse.
    
    Args:
        pdf_path (str): Pfad zur Bauausschreibungs-PDF
        floorplan_path (str): Pfad zum Grundriss (PDF/Bild)
        output_dir (str): Ausgabeverzeichnis
        
    Returns:
        dict: Ergebnisse des gesamten Workflows
    """
    
    # Ausgabeverzeichnis erstellen
    os.makedirs(output_dir, exist_ok=True)
    
    print("🏗️ === AUTOMATISIERTER BAU-WORKFLOW GESTARTET ===")
    print(f"📄 PDF: {pdf_path}")
    print(f"🏠 Grundriss: {floorplan_path}")
    print(f"📁 Ausgabe: {output_dir}")
    print()
    
    workflow_results = {
        'success': False,
        'text_extraction': None,
        'method_used': None,
        'svg_path': None,
        'extracted_info': None,
        'annotated_svg': None,
        'dataframes': None,
        'errors': []
    }
    
    try:
        # Schritt 1: Text aus PDF extrahieren
        print("🔍 Schritt 1: Text-Extraktion...")
        text, method = extract_text_from_pdf(pdf_path)
        
        if not text.strip():
            workflow_results['errors'].append("Keine Textextraktion möglich")
            print("❌ Keine Textextraktion möglich")
            return workflow_results
        
        workflow_results['text_extraction'] = text
        workflow_results['method_used'] = method
        print(f"✅ Text extrahiert mit Methode: {method}")
        print()
        
        # Schritt 2: Grundriss vektorisieren
        print("🎨 Schritt 2: Grundriss-Vektorisierung...")
        svg_path = vectorize_floorplan(floorplan_path, output_dir)
        
        if not svg_path:
            workflow_results['errors'].append("Vektorisierung fehlgeschlagen")
            print("❌ Vektorisierung fehlgeschlagen")
        else:
            workflow_results['svg_path'] = svg_path
            print(f"✅ Grundriss vektorisiert: {svg_path}")
        print()
        
        # Schritt 3: NLP-Informationsextraktion
        print("🧠 Schritt 3: NLP-Analyse...")
        extracted_info = extract_construction_information(text)
        
        if not extracted_info:
            workflow_results['errors'].append("NLP-Analyse fehlgeschlagen")
            print("❌ NLP-Analyse fehlgeschlagen")
        else:
            workflow_results['extracted_info'] = extracted_info
            # Strukturiere Daten in DataFrames
            dataframes = create_structured_dataframe(extracted_info)
            workflow_results['dataframes'] = dataframes
            print("✅ NLP-Analyse abgeschlossen")
        print()
        
        # Schritt 4: Automatische Annotation
        if svg_path and extracted_info:
            print("📝 Schritt 4: Automatische Annotation...")
            annotated_svg = annotate_floorplan_with_info(svg_path, extracted_info, 
                                                       os.path.join(output_dir, "annotated_floorplan.svg"))
            
            if annotated_svg:
                workflow_results['annotated_svg'] = annotated_svg
                print(f"✅ Annotation erstellt: {annotated_svg}")
            else:
                workflow_results['errors'].append("Annotation fehlgeschlagen")
                print("❌ Annotation fehlgeschlagen")
        else:
            print("⚠️ Schritt 4 übersprungen (Vektorisierung oder NLP fehlgeschlagen)")
        print()
        
        # Schritt 5: Ergebnisse speichern
        print("💾 Schritt 5: Ergebnisse speichern...")
        
        # Extrahierten Text speichern
        with open(os.path.join(output_dir, "extracted_text.txt"), 'w', encoding='utf-8') as f:
            f.write(text)
        
        # DataFrames als CSV speichern
        if workflow_results.get('dataframes'):
            for category, df in workflow_results['dataframes'].items():
                if not df.empty:
                    df.to_csv(os.path.join(output_dir, f"{category}.csv"), 
                             index=False, encoding='utf-8')
        
        # Workflow-Bericht erstellen
        report_path = os.path.join(output_dir, "workflow_report.txt")
        with open(report_path, 'w', encoding='utf-8') as f:
            f.write("AUTOMATISIERTER BAU-WORKFLOW BERICHT\n")
            f.write("=" * 50 + "\n\n")
            f.write(f"PDF-Datei: {pdf_path}\n")
            f.write(f"Grundriss: {floorplan_path}\n")
            f.write(f"Textextraktion: {method}\n")
            f.write(f"Textlänge: {len(text)} Zeichen\n")
            f.write(f"SVG erstellt: {'Ja' if svg_path else 'Nein'}\n")
            f.write(f"Annotation erstellt: {'Ja' if workflow_results.get('annotated_svg') else 'Nein'}\n")
            
            if extracted_info:
                f.write(f"\nExtrahierte Informationen:\n")
                f.write(f"- Materialien: {len(extracted_info.get('materials', []))}\n")
                f.write(f"- Räume: {len(extracted_info.get('rooms', []))}\n")
                f.write(f"- Maße: {len(extracted_info.get('measurements', []))}\n")
                f.write(f"- Positionen: {len(extracted_info.get('positions', []))}\n")
            
            if workflow_results['errors']:
                f.write(f"\nFehler: {', '.join(workflow_results['errors'])}\n")
        
        workflow_results['success'] = len(workflow_results['errors']) == 0
        
        print("✅ Workflow abgeschlossen!")
        print(f"📊 Bericht gespeichert: {report_path}")
        
        return workflow_results
        
    except Exception as e:
        workflow_results['errors'].append(str(e))
        print(f"❌ Workflow-Fehler: {e}")
        return workflow_results


In [40]:
# Einfacher Aufruf für kompletten Workflow
results = process_construction_documents(
    pdf_path="bauausschreibung.pdf",
    floorplan_path="grundriss.pdf", 
    output_dir="ergebnisse"
)

🏗️ === AUTOMATISIERTER BAU-WORKFLOW GESTARTET ===
📄 PDF: bauausschreibung.pdf
🏠 Grundriss: grundriss.pdf
📁 Ausgabe: ergebnisse

🔍 Schritt 1: Text-Extraktion...
❌ Workflow-Fehler: PDF-Datei nicht gefunden: bauausschreibung.pdf


#### Einfacher Aufruf des kompletten Workflows

---

## 🔍 Bonus-Workflow: Raster-PDF zu durchsuchbares PDF

Separater Workflow zur Konvertierung von gescannten/Raster-PDFs in durchsuchbare PDFs mit OCR-Text-Layer.


In [41]:
def convert_raster_pdf_to_searchable(input_pdf_path, output_pdf_path=None, 
                                     language='deu+eng', dpi=300):
    """
    Konvertiert eine Raster-PDF (gescannt) in eine durchsuchbare PDF mit OCR-Text-Layer.
    
    Args:
        input_pdf_path (str): Pfad zur Eingabe-PDF
        output_pdf_path (str): Pfad für Ausgabe-PDF (optional)
        language (str): Tesseract Sprachcode
        dpi (int): DPI für die Bildkonvertierung
        
    Returns:
        str: Pfad zur erstellten durchsuchbaren PDF
    """
    
    if not os.path.exists(input_pdf_path):
        print(f"❌ Eingabe-PDF nicht gefunden: {input_pdf_path}")
        return None
    
    # Ausgabepfad bestimmen
    if not output_pdf_path:
        base_name = os.path.splitext(input_pdf_path)[0]
        output_pdf_path = f"{base_name}_searchable.pdf"
    
    print(f"🔄 Konvertiere Raster-PDF zu durchsuchbarer PDF...")
    print(f"📄 Eingabe: {input_pdf_path}")
    print(f"💾 Ausgabe: {output_pdf_path}")
    
    try:
        # Schritt 1: Überprüfe ob bereits durchsuchbar
        if is_pdf_searchable(input_pdf_path):
            print("✅ PDF ist bereits durchsuchbar - kopiere Original")
            import shutil
            shutil.copy2(input_pdf_path, output_pdf_path)
            return output_pdf_path
        
        # Schritt 2: PDF zu Bildern konvertieren
        print("🖼️ Konvertiere PDF-Seiten zu Bildern...")
        images = convert_from_path(input_pdf_path, dpi=dpi)
        print(f"📋 {len(images)} Seiten gefunden")
        
        # Schritt 3: Einfachere Methode - Text extrahieren und neues PDF erstellen
        print("🔍 Starte OCR...")
        
        # Erstelle neues PDF-Dokument  
        new_doc = fitz.open()
        
        for page_num, image in enumerate(images):
            print(f"  📄 Verarbeite Seite {page_num + 1}/{len(images)}")
            
            # OCR für gesamten Text der Seite
            page_text = pytesseract.image_to_string(
                image, 
                lang=language,
                config='--psm 1'
            )
            
            # Neue Seite erstellen mit Original-Dimensionen
            page_width = image.width * 72 / dpi
            page_height = image.height * 72 / dpi
            new_page = new_doc.new_page(width=page_width, height=page_height)
            
            # Original-Bild einfügen
            import io
            img_byte_arr = io.BytesIO()
            image.save(img_byte_arr, format='PNG')
            img_bytes = img_byte_arr.getvalue()
            
            new_page.insert_image(fitz.Rect(0, 0, page_width, page_height), 
                                 stream=img_bytes)
            
            # Unsichtbaren Text hinzufügen (für Durchsuchbarkeit)
            if page_text.strip():
                # Text außerhalb des sichtbaren Bereichs platzieren
                new_page.insert_text(
                    (page_width + 10, 10),  # Außerhalb der Seite
                    page_text,
                    fontsize=1,  # Sehr kleine Schrift
                    color=(1, 1, 1),  # Weiß (unsichtbar)
                )
        
        # PDF speichern
        new_doc.save(output_pdf_path, garbage=4, deflate=True)
        new_doc.close()
        
        print(f"✅ Durchsuchbare PDF erstellt: {output_pdf_path}")
        
        # Validierung
        if is_pdf_searchable(output_pdf_path):
            print("✅ Validierung erfolgreich - PDF ist jetzt durchsuchbar!")
        else:
            print("⚠️ Warnung: PDF möglicherweise nicht vollständig durchsuchbar")
        
        return output_pdf_path
        
    except Exception as e:
        print(f"❌ Fehler bei der Konvertierung: {e}")
        return None


In [42]:
def batch_convert_pdfs_to_searchable(input_directory, output_directory=None):
    """
    Batch-Konvertierung mehrerer PDF-Dateien zu durchsuchbaren PDFs.
    
    Args:
        input_directory (str): Verzeichnis mit Eingabe-PDFs
        output_directory (str): Ausgabeverzeichnis (optional)
        
    Returns:
        dict: Ergebnisse der Batch-Konvertierung
    """
    
    if not os.path.exists(input_directory):
        print(f"❌ Eingabeverzeichnis nicht gefunden: {input_directory}")
        return None
    
    # Ausgabeverzeichnis bestimmen
    if not output_directory:
        output_directory = os.path.join(input_directory, "searchable_pdfs")
    
    os.makedirs(output_directory, exist_ok=True)
    
    # Finde alle PDF-Dateien
    pdf_files = [f for f in os.listdir(input_directory) if f.lower().endswith('.pdf')]
    
    if not pdf_files:
        print(f"❌ Keine PDF-Dateien in {input_directory} gefunden")
        return None
    
    print(f"📁 Batch-Konvertierung von {len(pdf_files)} PDF-Dateien")
    print(f"📥 Eingabe: {input_directory}")
    print(f"📤 Ausgabe: {output_directory}")
    print()
    
    results = {
        'total_files': len(pdf_files),
        'converted': [],
        'already_searchable': [],
        'failed': [],
        'processing_time': 0
    }
    
    import time
    start_time = time.time()
    
    for i, pdf_file in enumerate(pdf_files):
        print(f"🔄 [{i+1}/{len(pdf_files)}] Verarbeite: {pdf_file}")
        
        input_path = os.path.join(input_directory, pdf_file)
        output_filename = f"searchable_{pdf_file}"
        output_path = os.path.join(output_directory, output_filename)
        
        try:
            result_path = convert_raster_pdf_to_searchable(input_path, output_path)
            
            if result_path:
                if result_path == output_path:
                    results['converted'].append(pdf_file)
                else:
                    results['already_searchable'].append(pdf_file)
            else:
                results['failed'].append(pdf_file)
                
        except Exception as e:
            print(f"❌ Fehler bei {pdf_file}: {e}")
            results['failed'].append(pdf_file)
        
        print()  # Leerzeile zwischen Dateien
    
    results['processing_time'] = time.time() - start_time
    
    # Zusammenfassung
    print("=" * 60)
    print("📊 BATCH-KONVERTIERUNG ABGESCHLOSSEN")
    print("=" * 60)
    print(f"📄 Verarbeitete Dateien: {results['total_files']}")
    print(f"✅ Konvertiert: {len(results['converted'])}")
    print(f"✓ Bereits durchsuchbar: {len(results['already_searchable'])}")
    print(f"❌ Fehlgeschlagen: {len(results['failed'])}")
    print(f"⏱️ Verarbeitungszeit: {results['processing_time']:.1f} Sekunden")
    
    return results


In [ ]:
# Demo der PDF-zu-durchsuchbar Konvertierung
print("=== DEMO: RASTER-PDF ZU DURCHSUCHBARER PDF ===")

# Teste mit verfügbaren PDF-Dateien
test_files = ["Angebot_Schulzimmertuere.pdf"]

for test_file in test_files:
    if os.path.exists(test_file):
        print(f"\n🔍 Teste Datei: {test_file}")
        
        # Überprüfe aktuellen Status
        is_searchable = is_pdf_searchable(test_file)
        
        if not is_searchable:
            print("📝 Konvertiere zu durchsuchbarer PDF...")
            result = convert_raster_pdf_to_searchable(test_file)
            
            if result:
                print(f"✅ Durchsuchbare PDF erstellt: {result}")
            else:
                print("❌ Konvertierung fehlgeschlagen")
        else:
            print("✅ PDF ist bereits durchsuchbar")
        
        print("-" * 50)

# Beispiel für Batch-Konvertierung
print("\n💡 BATCH-KONVERTIERUNG BEISPIEL:")
print("# Konvertiere alle PDFs in einem Ordner:")
print("# results = batch_convert_pdfs_to_searchable('mein_pdf_ordner')")
print("\n💡 EINZELNE DATEI BEISPIEL:")
print("# result = convert_raster_pdf_to_searchable('gescannte_datei.pdf')")

print("\n🎯 NUTZUNGSHINWEISE:")
print("• Funktioniert mit gescannten PDFs (Raster-Bildern)")
print("• Erkennt automatisch ob PDF bereits durchsuchbar ist")
print("• Behält Original-Bildqualität bei")
print("• Fügt unsichtbaren Text-Layer für Suchfunktion hinzu")
print("• Unterstützt Deutsch + Englisch OCR")
print("• Batch-Modus für mehrere Dateien verfügbar")


=== DEMO: RASTER-PDF ZU DURCHSUCHBARER PDF ===

🔍 Teste Datei: Angebot_Schulzimmertuere.pdf
📄 PDF-Status: Durchsuchbar
📝 Gefundener Text: 1984 Zeichen
✅ PDF ist bereits durchsuchbar
--------------------------------------------------

💡 BATCH-KONVERTIERUNG BEISPIEL:
# Konvertiere alle PDFs in einem Ordner:
# results = batch_convert_pdfs_to_searchable('mein_pdf_ordner')

💡 EINZELNE DATEI BEISPIEL:
# result = convert_raster_pdf_to_searchable('gescannte_datei.pdf')

🎯 NUTZUNGSHINWEISE:
• Funktioniert mit gescannten PDFs (Raster-Bildern)
• Erkennt automatisch ob PDF bereits durchsuchbar ist
• Behält Original-Bildqualität bei
• Fügt unsichtbaren Text-Layer für Suchfunktion hinzu
• Unterstützt Deutsch + Englisch OCR
• Batch-Modus für mehrere Dateien verfügbar


In [44]:
# Einzelne Datei
result = convert_raster_pdf_to_searchable('gescannte_datei.pdf')

# Batch-Verarbeitung
results = batch_convert_pdfs_to_searchable('mein_pdf_ordner')

❌ Eingabe-PDF nicht gefunden: gescannte_datei.pdf
❌ Eingabeverzeichnis nicht gefunden: mein_pdf_ordner


#### BKP

In [2]:
import pandas as pd

# Pfad zur heruntergeladenen CSV-Datei angeben
csv_path = "BKP-Plan.csv"  # Passe den Dateinamen an, falls du anders gespeichert hast

# CSV in DataFrame laden
df_bkp = pd.read_csv(csv_path)

# Ersten Überblick verschaffen
#df_bkp.head(30)
df_bkp

,Position,Bezeichnung,Kategorie
0,0,Übergangspositionen,0 Grundstück > 00 Vorstudien
1,1,Studien zur Grundstückbeurteilung,0 Grundstück > 00 Vorstudien
2,2,"Vermessung, Vermarchung",0 Grundstück > 00 Vorstudien
3,3,Geotechnische Gutachten,0 Grundstück > 00 Vorstudien
4,4,Quartierplankosten,0 Grundstück > 00 Vorstudien
...,...,...,...
531,992,993,9 Ausstatung > 99 Honorare
532,994,995,9 Ausstatung > 99 Honorare
533,996,Spezialisten,9 Ausstatung > 99 Honorare
534,997,998,9 Ausstatung > 99 Honorare
